In [1]:
!pip uninstall -y jax jaxlib
!pip install -Uqq optuna kaleido datasets

Found existing installation: jax 0.4.26
Uninstalling jax-0.4.26:
  Successfully uninstalled jax-0.4.26
Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
  Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata, runtime
import subprocess

In [3]:
hf_token = userdata.get('hf_token')
input_str = f'{hf_token}\nn\n'
result = subprocess.run(['huggingface-cli', 'login'], input=input_str, text=True, capture_output=True)
print(result.stdout)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Add token as git credential? (Y/n) Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful



In [ ]:
from functools import reduce
import os
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, default_collate
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, transforms, CenterCrop, RandomCrop
from torchvision.transforms.v2 import CutMix
from sklearn.metrics import f1_score
import optuna
import pickle
import json

from datasets import load_dataset, concatenate_datasets

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

results_dir = '/content/drive/MyDrive/Colab_Notebooks/dye_test_opt/ternary/results/learning'

n_trials = 150

# Load dataset
ds = load_dataset('mpg-ranch/dye_test', split='train')

# Dataset preparation
total_samples = len(ds)
examples_per_set = 88 #multiplied by 3x months later
n_workers = os.cpu_count()
print(f'Number of available CPU cores: {n_workers}')
n_epochs = 10
n_seeds = 3
context_sz = 154

imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                  'std': [0.229, 0.224, 0.225]}

# Preprocessing transforms
preprocs = Compose([
    CenterCrop((context_sz, context_sz)),
])

def preproc_transforms(examples):
    examples["img"] = [preprocs(image.convert("RGB")) for image in examples["image"]]
    return examples

print("Applying preprocessing transforms...")
ds = ds.map(preproc_transforms, remove_columns=["image","color","size","concentration"], batched=True, batch_size=len(ds))
n_classes = len(np.unique(ds['label']))

def filter_month(batch, month):
    """Returns a list of booleans indicating whether each example in the batch should be kept."""
    # Assuming 'month' is a key in your dataset's dictionary and it's stored in a list for each batch
    return [example_month == month for example_month in batch['month']]

# Define model loading function
def load_model(arch, n_classes):
    print("Loading model...")
    model = torch.hub.load('facebookresearch/dinov2', arch)
    num_ftrs = model.norm.normalized_shape[0]
    model.head = nn.Linear(num_ftrs, n_classes)
    model.to(device)
    return model

def modified_f1_score(labels, predictions):
    scores = []

    # For 1 vs. 0, exclude class 2 and consider only 1 and 0 for binary comparison
    mask_1_vs_0 = (labels != 2)
    labels_1_vs_0 = labels[mask_1_vs_0] == 1
    predictions_1_vs_0 = predictions[mask_1_vs_0] == 1
    if np.any(labels_1_vs_0) or np.any(predictions_1_vs_0):
        f1_1_vs_0 = f1_score(labels_1_vs_0, predictions_1_vs_0, pos_label=True, average='binary')
        scores.append(f1_1_vs_0)

    # For 2 vs. 0, exclude class 1 and consider only 2 and 0 for binary comparison
    mask_2_vs_0 = (labels != 1)
    labels_2_vs_0 = labels[mask_2_vs_0] == 2
    predictions_2_vs_0 = predictions[mask_2_vs_0] == 2
    if np.any(labels_2_vs_0) or np.any(predictions_2_vs_0):
        f1_2_vs_0 = f1_score(labels_2_vs_0, predictions_2_vs_0, pos_label=True, average='binary')
        scores.append(f1_2_vs_0)

    # Calculate the mean of the F1 scores if any valid scores were calculated
    mean_f1 = np.mean(scores) if scores else 0.0

    return f1_1_vs_0, f1_2_vs_0, mean_f1

# Define Optuna objective function
def objective(trial):
    print(f"Starting trial: {trial.number}")

    # Hyperparameters to tune
    lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
    bs = trial.suggest_int('bs', 16, 256, step=16)

    datasets_by_month = {
        month: ds.filter(function=filter_month, batched=True, batch_size=len(ds), fn_kwargs={'month': month})
        for month in ['may', 'july', 'october']
    }

    f1_scores = {epoch: [] for epoch in range(1, n_epochs + 1)}

    # Seed loop
    for seed in range(n_seeds):
        print(f"Seed: {seed}")
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        train_ds_list = []
        val_ds_list = []

        # Sample examples_per_set for training and validation from each month's dataset
        for month, dataset in datasets_by_month.items():
            total_samples = len(dataset)
            all_indices = list(range(total_samples))
            sampled_indices = random.sample(all_indices, 2 * examples_per_set)
            train_indices = sampled_indices[:examples_per_set]
            val_indices = sampled_indices[examples_per_set:examples_per_set*2]

            train_ds_month = dataset.select(train_indices)
            val_ds_month = dataset.select(val_indices)

            train_ds_list.append(train_ds_month)
            val_ds_list.append(val_ds_month)

        # Combine the training and validation sets across months
        train_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), train_ds_list)
        val_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), val_ds_list)

        # Model, optimizer, and loss function setup
        model = load_model('dinov2_vitb14', n_classes)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss().to(device)

        tfms = Compose([
            CenterCrop(size=context_sz),
            ToTensor(),
            Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'])  # Normalize using ImageNet stats
        ])

        def batch_tfms(examples):
            examples["img"] = [tfms(image) for image in examples["img"]]
            return examples

        train_ds_combined.set_transform(batch_tfms)
        val_ds_combined.set_transform(batch_tfms)

        train_loader = DataLoader(train_ds_combined, batch_size=bs, shuffle=True, num_workers=n_workers)
        val_loader = DataLoader(val_ds_combined, batch_size=bs, num_workers=n_workers)

        # Training loop
        for epoch in range(1, n_epochs +1):
            print(f"Epoch: {epoch}/{n_epochs}")
            model.train()
            for _, data in enumerate(train_loader):
                inputs, labels = data['img'].to(device), data['label'].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            # Validation loop
            model.eval()
            all_labels, all_predictions = [], []
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['img'].to(device), data['label'].to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())

            # Compute F1 score
            blue, red, f1 = modified_f1_score(np.array(all_labels), np.array(all_predictions))
            print(f"Blue F1 score: {blue:.4f}\nRed F1 score: {red:.4f}\nMean F1 score: {f1:.4f}")

            f1_scores[epoch].append(f1)

    # Return the mean F1 score over the seeds
    mean_f1_scores = {epoch: np.mean(scores) for epoch, scores in f1_scores.items()}
    highest_f1_epoch, highest_f1_score = max(mean_f1_scores.items(), key=lambda x: x[1])

    print(f"Highest F1-score {highest_f1_score} at epoch {highest_f1_epoch}")

    return highest_f1_score

def study_cb(study, trial):
    patience = 50

    # Directory where the figures and trial data will be saved
    os.makedirs(results_dir, exist_ok=True)

    # Save figures
    if trial.state == optuna.trial.TrialState.COMPLETE:
        opt_hist_path = os.path.join(results_dir, f'history_plot.png')
        fig1 = optuna.visualization.plot_optimization_history(study)
        fig1.write_image(opt_hist_path)

        opt_slice_path = os.path.join(results_dir, f'slice_plot.png')
        fig2 = optuna.visualization.plot_slice(study)
        fig2.write_image(opt_slice_path)

    study_path = os.path.join(results_dir, 'study.pkl')
    with open(study_path, 'wb') as f:
        pickle.dump(study, f)

    if 'best_trial_number' not in study.user_attrs:
        # Initialize with the first trial
        study.set_user_attr('best_trial_number', 0)

    current_best_trial = study.best_trial.number
    best_trial_number = study.user_attrs['best_trial_number']

    # Update the best trial number in study attributes if the current best is newer
    if current_best_trial > best_trial_number:
        study.set_user_attr('best_trial_number', current_best_trial)
        best_trial_number = current_best_trial

    # Check if no improvement has been made for 'tolerance' trials
    if trial.number >= best_trial_number + patience:
        study.stop()


study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective, n_trials=n_trials, callbacks=[study_cb])

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Number of available CPU cores: 12
Applying preprocessing transforms...


...

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Epoch: 1/10
Blue F1 score: 0.7750
Red F1 score: 0.8642
Mean F1 score: 0.8196
Epoch: 2/10
Blue F1 score: 0.8052
Red F1 score: 0.9351
Mean F1 score: 0.8701
Epoch: 3/10
Blue F1 score: 0.8205
Red F1 score: 0.9444
Mean F1 score: 0.8825
Epoch: 4/10
Blue F1 score: 0.8333
Red F1 score: 0.9444
Mean F1 score: 0.8889
Epoch: 5/10
Blue F1 score: 0.8434
Red F1 score: 0.9067
Mean F1 score: 0.8750
Epoch: 6/10
Blue F1 score: 0.8298
Red F1 score: 0.8132
Mean F1 score: 0.8215
Epoch: 7/10
Blue F1 score: 0.8132
Red F1 score: 0.9444
Mean F1 score: 0.8788
Epoch: 8/10
Blue F1 score: 0.8250
Red F1 score: 0.9444
Mean F1 score: 0.8847
Epoch: 9/10
Blue F1 score: 0.8354
Red F1 score: 0.9444
Mean F1 score: 0.8899
Epoch: 10/10
Blue F1 score: 0.8354
Red F1 score: 0.9444
Mean F1 score: 0.8899
Seed: 2
Loading model...


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Epoch: 1/10
Blue F1 score: 0.5517
Red F1 score: 0.5484
Mean F1 score: 0.5501
Epoch: 2/10
Blue F1 score: 0.8056
Red F1 score: 0.8101
Mean F1 score: 0.8078
Epoch: 3/10
Blue F1 score: 0.8333
Red F1 score: 0.8421
Mean F1 score: 0.8377
Epoch: 4/10
Blue F1 score: 0.6406
Red F1 score: 0.7429
Mean F1 score: 0.6917
Epoch: 5/10
Blue F1 score: 0.7042
Red F1 score: 0.8108
Mean F1 score: 0.7575
Epoch: 6/10
Blue F1 score: 0.7606
Red F1 score: 0.8537
Mean F1 score: 0.8071
Epoch: 7/10
Blue F1 score: 0.6970
Red F1 score: 0.8267
Mean F1 score: 0.7618
Epoch: 8/10
Blue F1 score: 0.7353
Red F1 score: 0.8571
Mean F1 score: 0.7962
Epoch: 9/10
Blue F1 score: 0.7353
Red F1 score: 0.8421
Mean F1 score: 0.7887
Epoch: 10/10


[I 2024-04-12 08:12:30,153] Trial 99 finished with value: 0.8772898407043394 and parameters: {'lr': 1.1370194180641989e-05, 'bs': 16}. Best is trial 49 with value: 0.897237345020408.


In [ ]:
runtime.unassign()